In [1]:
import xarray as xr
import xroms

In [2]:
A = xroms.roms_dataset('ocean_avg_0014.nc')

In [3]:
temp = A.temp

In [4]:
t = 2
s = -1    # Surface
i = 100
j = 110
temp[t, s, j, i]

<xarray.DataArray 'temp' ()>
array(9.756377220153809, dtype=float32)
Coordinates:
    xi_rho   int64 100
    eta_rho  int64 110
    time     datetime64[ns] 1989-05-30T12:00:00
    s_rho    float64 -0.01562
    z_rho    float32 -0.594677
    lat_rho  float64 59.7
    lon_rho  float64 1.189
Attributes:
    long_name:  averaged potential temperature
    units:      Celsius
    time:       ocean_time
    field:      temperature, scalar, series

In [5]:
# Do not need to remember the order of the indices
temp.isel(xi_rho=i, eta_rho=j, s_rho=-1, time=2)

<xarray.DataArray 'temp' ()>
array(9.756377220153809, dtype=float32)
Coordinates:
    xi_rho   int64 100
    eta_rho  int64 110
    time     datetime64[ns] 1989-05-30T12:00:00
    s_rho    float64 -0.01562
    z_rho    float32 -0.594677
    lat_rho  float64 59.7
    lon_rho  float64 1.189
Attributes:
    long_name:  averaged potential temperature
    units:      Celsius
    time:       ocean_time
    field:      temperature, scalar, series

In [6]:
# Can use index values
SST = A['temp'].isel(s_rho=-1)
float(SST.sel(xi_rho=i, eta_rho=j, time='1989-05-30 12'))

9.756377220153809

In [7]:
# The xi_u coordinate shows the stagerring
# For u-velocity
u0 = A.u[t, s, j, i]
print(float(u0))
u1 = A.u.isel(time=t, s_rho=s).sel(xi_u=i+0.5, eta_u=j)
print(float(u1))


-0.08350111544132233
-0.08350111544132233


In [8]:
# Take a subgrid
i0, i1, j0, j1 = 10, 110, 30, 120
B = xroms.subgrid(A, (i0, i1, j0, j1))
B.dims

Frozen(SortedKeysDict(OrderedDict([('eta_rho', 91), ('xi_rho', 101), ('time', 10), ('s_rho', 32), ('eta_u', 91), ('xi_u', 102), ('eta_v', 92), ('xi_v', 101), ('s_w', 33)])))

In [9]:
# Numerical indexing is offset by subgrid
print(float(A.temp[t, s, j, i]))
print(float(B.temp[t, s, j-j0, i-i0]))

9.756377220153809
9.756377220153809


In [10]:
# Indexing by coordinates are invariant under subgrid
print(float(A.temp[t, s].sel(xi_rho=i, eta_rho=j)))
print(float(B.temp[t, s].sel(xi_rho=i, eta_rho=j)))

9.756377220153809
9.756377220153809


In [11]:
# Check velocity (burde ikke disse være like)
print(float(A.u[t, s, j, i]))
print(float(B.u[t, s, j-j0, i-i0]))    # Må justere litt hva indeks skal være
print(float(B.u[t, s, j-j0, i-i0+1]))  # Dette er den riktige  

# Coordinate based velocity works just fine
print(float(A.u[t, s].sel(xi_u=i+0.5, eta_u=j)))
print(float(B.u[t, s].sel(xi_u=i+0.5, eta_u=j)))

# Problemet er at hvis A er hele, har ikke hastighet på randen.
# men har det hvis subgrid.
# En mulig løsning: lese hele, ikke ta med rho-punkter på randen.
# Alternativ løsning: subgrid, ikke ta med ytterste u, v punkter.
# Begge metoder => hiver vekk informasjon
                        

-0.08350111544132233
-0.08275257796049118
-0.08350111544132233
-0.08350111544132233
-0.08350111544132233
